<a href="https://colab.research.google.com/github/Justholli/labs/blob/main/1lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для русского языка подберите 5 различных спикеров (например, на YouTube). Для скачивания с YT отправлял ссылку выше
Возьмите для каждого по 2 фрагмента речи длительностью 10-20 секунд.
Проведите распознавание речи для фрагментов с помощью Whisper на модели base и large-v2
Также сделайте транскрипцию самостоятельно, прослушав семплы.
Посчитайте метрики Word Error Rate и Character Error Rate для этих
фрагментов. За эталон возьмите вашу транскрипцию, считайте метрики для Whisper base и large-v2)

**Загрузим библиотеки для работы, для подсчитывания WER и CER будем использовать jiwer и Levenshtein**

In [1]:
!pip install pydub
!pip install jiwer
!pip install python-Levenshtein
!pip install transformers
!pip install git+https://github.com/openai/whisper.git -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.9 MB/s eta 0:00:00


In [2]:
from pathlib import Path
from IPython.display import Audio
from pydub import AudioSegment
import jiwer
import Levenshtein
import numpy as np
from transformers import pipeline
from google.colab import drive
import whisper


In [3]:
baseModel = whisper.load_model("base")
largeV2Model = whisper.load_model("large-v2")


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 94.4MiB/s]
100%|█████████████████████████████████████| 2.87G/2.87G [00:31<00:00, 98.5MiB/s]


**Спикер номер 1**

In [13]:
audio1_1 = whisper.load_audio("/content/sample_data/123.mp4")
trans1_1 = "Ребята, всем привет! Сегодня я расскажу как установить Android Studio, откуда, какую версию и как настроить первый проект."
audio1_2 = whisper.load_audio("/content/sample_data/234.mp4")
trans1_2 = "вот каким образом это получилось. Здесь есть вкладочка другие версии вот и мы можем установить их"

In [5]:
audio1_1 = whisper.pad_or_trim(audio1_1)
audio1_2 = whisper.pad_or_trim(audio1_2)
mel1_1B = whisper.log_mel_spectrogram(audio1_1).to(baseModel.device)
mel1_1L = whisper.log_mel_spectrogram(audio1_1).to(largeV2Model.device)
mel1_2B = whisper.log_mel_spectrogram(audio1_2).to(baseModel.device)
mel1_2L = whisper.log_mel_spectrogram(audio1_2).to(largeV2Model.device)

In [6]:
options = whisper.DecodingOptions()
result1_1B = whisper.decode(baseModel, mel1_1B, options)
result1_1L = whisper.decode(largeV2Model, mel1_1L, options)
result1_2B = whisper.decode(baseModel, mel1_2B, options)
result1_2L = whisper.decode(largeV2Model, mel1_2L, options)

In [7]:
print(result1_1B.text)
print(result1_1L.text)
print(result1_2B.text)
print(result1_2L.text)

Ребята, всем привет! Сегодня расскажу, как установить анорой студию, откуда, какой версию и как настроить первый проект.
Ребята, всем привет! Сегодня я расскажу как установить Android Studio, откуда, какую версию и как настроить первый проект.
Вот каким образом это получилось. Здесь есть вкладка другие версии. Вот и мы можем установить их.
вот каким образом это получилось. здесь есть вкладочка другие версии вот и мы можем установить их


In [14]:
wer1_1B = jiwer.wer(trans1_1, result1_1B.text)
print(f"Word Error Rate for Text 1 with Base: {wer1_1B}")
wer1_1L = jiwer.wer(trans1_1, result1_1L.text)
print(f"Word Error Rate for Text 1 with Large-v2: {wer1_1L}")
wer1_2B = jiwer.wer(trans1_2, result1_2B.text)
print(f"Word Error Rate for Text 2 with Base: {wer1_2B}")
wer1_2L = jiwer.wer(trans1_2, result1_2L.text)
print(f"Word Error Rate for Text 2 with Large-v2: {wer1_2L}")

Word Error Rate for Text 1 with Base: 0.2777777777777778
Word Error Rate for Text 1 with Large-v2: 0.0
Word Error Rate for Text 2 with Base: 0.3125
Word Error Rate for Text 2 with Large-v2: 0.0625


In [15]:
cer1_1B = Levenshtein.distance(trans1_1, result1_1B.text) / len(trans1_1)
print(f"Character Error Rate for Text 1 with Base: {cer1_1B}")
cer1_1L = Levenshtein.distance(trans1_1, result1_1L.text) / len(trans1_1)
print(f"Character Error Rate for Text 1 with Large-v2: {cer1_1L}")
cer1_2B = Levenshtein.distance(trans1_2, result1_2B.text) / len(trans1_2)
print(f"Character Error Rate for Text 2 with Base: {cer1_2B}")
cer1_2L = Levenshtein.distance(trans1_2, result1_2L.text) / len(trans1_2)
print(f"Character Error Rate for Text 2 with Large-v2: {cer1_2L}")

Character Error Rate for Text 1 with Base: 0.14754098360655737
Character Error Rate for Text 1 with Large-v2: 0.0
Character Error Rate for Text 2 with Base: 0.061855670103092786
Character Error Rate for Text 2 with Large-v2: 0.010309278350515464


**Спикер номер 2**

In [19]:
audio2_1 = whisper.load_audio("/content/sample_data/345.mp4")
audio2_2 = whisper.load_audio("/content/sample_data/456.mp4")
trans2_1 = "Я попробовал больше 50 разных ноукод сервисов. И в этом видео я расскажу про лучшие из них. Это сервисы, в которых"
trans2_2 = "приложение без разработчика, без кода и в кратчайшее время. В каждом из этих сервисов есть свои нюансы, поэтому посмотрите видео полностью, чтобы"
audio2_1 = whisper.pad_or_trim(audio2_1)
audio2_2 = whisper.pad_or_trim(audio2_2)
mel2_1B = whisper.log_mel_spectrogram(audio2_1).to(baseModel.device)
mel2_1L = whisper.log_mel_spectrogram(audio2_1).to(largeV2Model.device)
mel2_2B = whisper.log_mel_spectrogram(audio2_2).to(baseModel.device)
mel2_2L = whisper.log_mel_spectrogram(audio2_2).to(largeV2Model.device)

In [18]:
result2_1B = whisper.decode(baseModel, mel2_1B, options)
result2_1L = whisper.decode(largeV2Model, mel2_1L, options)
result2_2B = whisper.decode(baseModel, mel2_2B, options)
result2_2L = whisper.decode(largeV2Model, mel2_2L, options)
print(result2_1B.text)
print(result2_1L.text)
print(result2_2B.text)
print(result2_2L.text)

Я попробовал больше 50 разных ноукод сервисов. И в этом видео я расскажу про лучшие из них. Это сервисы, в которых
Я попробовал больше 50 разных наукодсервисов и в этом видео я расскажу про лучшие из них.
приложение без разработчиков, без кодов и в кратчешие время. В каждом из этих сервисов есть свои нюансы, поэтому посмотрите видео полностью, чтобы
приложение без разработчика, без кода и в кратчайшее время. В каждом из этих сервисов есть свои нюансы, поэтому посмотрите видео полностью, чтобы


In [20]:
wer2_1B = jiwer.wer(trans2_1, result2_1B.text)
print(f"Word Error Rate for Text 1 with Base: {wer2_1B}")
wer2_1L = jiwer.wer(trans2_1, result2_1L.text)
print(f"Word Error Rate for Text 1 with Large-v2: {wer2_1L}")
wer2_2B = jiwer.wer(trans2_2, result2_2B.text)
print(f"Word Error Rate for Text 2 with Base: {wer2_2B}")
wer2_2L = jiwer.wer(trans2_2, result2_2L.text)
print(f"Word Error Rate for Text 2 with Large-v2: {wer2_2L}")
cer2_1B = Levenshtein.distance(trans2_1, result2_1B.text) / len(trans2_1)
print(f"Character Error Rate for Text 1 with Base: {cer2_1B}")
cer2_1L = Levenshtein.distance(trans2_1, result2_1L.text) / len(trans2_1)
print(f"Character Error Rate for Text 1 with Large-v2: {cer2_1L}")
cer2_2B = Levenshtein.distance(trans2_2, result2_2B.text) / len(trans2_2)
print(f"Character Error Rate for Text 2 with Base: {cer2_2B}")
cer2_2L = Levenshtein.distance(trans2_2, result2_2L.text) / len(trans2_2)
print(f"Character Error Rate for Text 2 with Large-v2: {cer2_2L}")

Word Error Rate for Text 1 with Base: 0.0
Word Error Rate for Text 1 with Large-v2: 0.3333333333333333
Word Error Rate for Text 2 with Base: 0.13636363636363635
Word Error Rate for Text 2 with Large-v2: 0.0
Character Error Rate for Text 1 with Base: 0.0
Character Error Rate for Text 1 with Large-v2: 0.23684210526315788
Character Error Rate for Text 2 with Base: 0.04827586206896552
Character Error Rate for Text 2 with Large-v2: 0.0


**Спикер номер 3**

In [23]:
audio3_1 = whisper.load_audio("/content/sample_data/567.mp4")
audio3_2 = whisper.load_audio("/content/sample_data/678.mp4")
trans3_1 = "Камары, муха в глаз попала. Здрасте! С вами Даня Краестера, это канал «Дикий Краестер». Мы сегодня будем учиться на"
trans3_2 = "Та-та-та-та-та-та, это «Дикий Крастер» та-та-та ву-ху та-та-та-та-та. Прежде чем мы начнём, напоминаю, что есть телеграм-канал Злой Крастер и страница на"
audio3_1 = whisper.pad_or_trim(audio3_1)
audio3_2 = whisper.pad_or_trim(audio3_2)
mel3_1B = whisper.log_mel_spectrogram(audio3_1).to(baseModel.device)
mel3_1L = whisper.log_mel_spectrogram(audio3_1).to(largeV2Model.device)
mel3_2B = whisper.log_mel_spectrogram(audio3_2).to(baseModel.device)
mel3_2L = whisper.log_mel_spectrogram(audio3_2).to(largeV2Model.device)

result3_1B = whisper.decode(baseModel, mel3_1B, options)
result3_1L = whisper.decode(largeV2Model, mel3_1L, options)
result3_2B = whisper.decode(baseModel, mel3_2B, options)
result3_2L = whisper.decode(largeV2Model, mel3_2L, options)
print(result3_1B.text)
print(result3_1L.text)
print(result3_2B.text)
print(result3_2L.text)

wer3_1B = jiwer.wer(trans3_1, result3_1B.text)
print(f"Word Error Rate for Text 1 with Base: {wer3_1B}")
wer3_1L = jiwer.wer(trans3_1, result3_1L.text)
print(f"Word Error Rate for Text 1 with Large-v2: {wer3_1L}")
wer3_2B = jiwer.wer(trans3_2, result3_2B.text)
print(f"Word Error Rate for Text 2 with Base: {wer3_2B}")
wer3_2L = jiwer.wer(trans3_2, result3_2L.text)
print(f"Word Error Rate for Text 2 with Large-v2: {wer3_2L}")
cer3_1B = Levenshtein.distance(trans3_1, result3_1B.text) / len(trans3_1)
print(f"Character Error Rate for Text 1 with Base: {cer3_1B}")
cer3_1L = Levenshtein.distance(trans3_1, result3_1L.text) / len(trans3_1)
print(f"Character Error Rate for Text 1 with Large-v2: {cer3_1L}")
cer3_2B = Levenshtein.distance(trans3_2, result3_2B.text) / len(trans3_2)
print(f"Character Error Rate for Text 2 with Base: {cer3_2B}")
cer3_2L = Levenshtein.distance(trans3_2, result3_2L.text) / len(trans3_2)
print(f"Character Error Rate for Text 2 with Large-v2: {cer3_2L}")

Камары, ухо в глаз попал. Здрасте! С вами Даня Краестера, это канал «Дикий Краестер». Мы сегодня будем учиться на
Комары, ухо в глаз попало. Здрасте, с вами Даня Крастер, это канал Дикий Крастер. Мы сегодня будем учиться на...
Те-кететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететететет
Прежде чем мы начнём, напоминаю, что есть телеграм-канал Злой Крастер и страница на
Word Error Rate for Text 1 with Base: 0.10526315789473684
Word Error Rate for Text 1 with Large-v2: 0.47368421052631576
Word Error Rate for Text 2 with Base: 1.0
Word Error Rate for Text 2 with Large-v2: 0.35
Character Error Rate for Text 1 with 

Тут немного в музыкальном стиле, поэтому моделькам тяжело)

Как видно по итогам, в большинстве случаев распознавание от модели large-v2 получалось точнее. Это неудивительно, тк она тяжеловеснее чем base, тк обучалась на большем объеме данных. Но соответственно base легковеснее и удобнее для небольших задач.